In [1]:
from sklearn.model_selection import train_test_split

In [2]:
x_set = []
y_set = []

with open("dialog_acts.dat", 'r') as file:
    for line in file:
        y_set.append(line.split()[0])
        x_set.append(" ".join(line.split()[1:]))

y_set[0], x_set[0]
    

('inform', 'im looking for a moderately priced restaurant that serves')

In [16]:
# x_train, x_test = x_set[int(len(x_set)*0.15):], x_set[:int(len(x_set)*0.15)]
# y_train, y_test = y_set[int(len(y_set)*0.15):], y_set[:int(len(y_set)*0.15)]
# len(x_train), len(x_test), len(y_train), len(y_test)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(x_set, y_set, test_size=0.15, random_state=42)
len(X_train), len(X_test), len(y_train), len(y_test)


(21675, 3826, 21675, 3826)

## Dummy baseline

In [4]:
def dummy_baseline(query: str) -> str:
    return max(set(y_train), key=y_train.count)

In [19]:
print(dummy_baseline('hi'))

inform


In [10]:
dummy_baseline_acc = y_test.count('inform') / len(y_test)
print(dummy_baseline_acc)

0.40041819132253004


## Rule-based baseline : TODO

In [169]:
# Function takes as arguments a sentence and a category-keyword dictionary.

def rule_based_baseline(sentence, keyword_dict):
    result = [0] * 15
    count = [0] * 15
    words = sentence.split()
    i = 0
    for keywords in keyword_dict.values(): # Iterating over categories
        for key in keywords: # Iterating over keywords in the categories
            for word in words: # Iterating over words in the sentence
                if key == word:
                    count[i] += 1
        i += 1
    max_keywards = max(count) # Finding number of the most maching keywords within a category
    index_of_max_keywords =  count.index(max_keywards) # Finding the index of the category with the most matching keywords
    result[index_of_max_keywords] = 1 # Encoding the categry 
    return result # Function returns a encoded category, encoding matches what a neural network would return.
    # This is done for ease of inegration with other systems that may be build during this project.

In [175]:
# Category-keyword dictionary

keyword_dict = {
    'ack': ['kay', 'okay', 'good', 'fine'],
    'affirm': ['yes', 'right', 'correct', 'yeah', 'ye', 'right', 'correct', 'perfect'],
    'bye': ['good', 'bye'],
    'confirm': ['does', 'is', 'it'],
    'deny': ['wrong', 'want', 'dont'],
    'hello': ['hi', 'hello', 'im', 'looking'],
    'inform': ['any', 'food', 'dont', 'care', 'expensive', 'moderate', 'cheap', 'east', 'west', 'north', 'south', 'centre', 'town', 'area', 'im', 'need', 'restaurant', 'looking'],
    'negate': ['no'],
    'null': ['unintelligible', 'noise', 'what', 'uh', 'sil', 'laughing'],
    'repeat': ['repeat', 'back', 'go', 'again'],
    'reqalts': ['else', 'next', 'how', 'about', 'any', 'anything', 'is', 'there', 'other'],
    'reqmore': ['more'],
    'request': ['type', 'phone', 'number', 'address', 'post', 'code', 'could', 'what', 'is', 'the', 'type', 'whats', 'may', 'i'],
    'restart': ['start', 'over', 'reset'],
    'thankyou': ['thank', 'you', 'good', 'bye', 'goodbye'],
}

In [171]:
# Category-code dictionary

result_dict = {
    'ack': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'affirm': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'bye': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'confirm': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'deny': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'hello': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'inform': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    'negate': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    'null': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    'repeat': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    'reqalts': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    'reqmore': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    'request': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    'restart': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    'thankyou': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
}

In [174]:
# Making predictions and calculating the success reate

positive = 0
for x in range(len(X_test)): # Iterating over sentences in the set
    pred_coded = rule_based_baseline(X_test[x], keyword_dict) # Callin the predicting function to get the coded predicted category
    for key, value in result_dict.items(): # Decoding categories
        if pred_coded == value:
            pred = key
    if pred == y_test[x]: # Adding 1 to successful predition count if the prediction matches data
        positive += 1
rate = positive / len(y_test) # Calcualting and displaying a success rate
print('Rate is: ', rate)

Rate:  0.8429168844746472
